In [18]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
import time

### Taking in the data

In [20]:
data = []
ans = 0
with open("./data/train.txt", "r", encoding="utf-8") as f:
    tweet = {}
    for row in f.readlines():
        row_data = row.strip().split()
        if len(row_data)==0 or len(row_data)==1:
            continue
        if row_data[0]=="meta":
            data.append(tweet)
            tweet = {}
            tweet["id"] = row_data[1]
            tweet["class"] = row_data[2]
            tweet["text"] = []
            tweet["token_type"] = []
        else:
            tweet["text"].append(row_data[0])
            tweet["token_type"].append(row_data[1])
data = data[1:]

## Preprocessing functions

In [35]:
# Returns True if anything other than A-Z or a-z is present in string
def extrachar(word):
    for ch in word:
        o = ord(ch)
        if (o>122 or o<65):
            return True
    return False

# Cleans the tweets by removing useless tokens, links, mentions and hashtags
def clean_tweet(tweet):
    dat = zip(tweet["text"], tweet["token_type"])
    li = []
    dat2 = []
    flag = 0
    for i,t in dat:
        if(flag==1):
            flag = 0
            continue
        if(i=="_" or i=="@" or i=="#"):
            flag = 1
            continue
        dat2.append((i,t))
    dat = dat2
    lx = lambda x: x[1]!="O"
    lt = map(lambda x: (x[0].lower(), x[1]),filter(lx,dat))
    for i,t in lt:
        if(i=="http" or i=="https" or i=="t"):
            break
        else:
            if((not (extrachar(i) or len(i)<3)) and t!="EMT"):
                li.append((i,t))
            if(t=="EMT"):
                li.append((i,t))
    ttext = [x[0] for x in li]
    ttoken = [x[1] for x in li]
    return ttext, ttoken

### Prepares list of Input and Sentiment

In [32]:
inp = []
outp = []
ttype = []
in_vocab = set()
ttype_vocab = set(["Hin", "Eng", "EMT"])
out_vocab = set(["neutral", "positive", "negative"])

for tweet in data:
    if(tweet["class"] in ["negative", "neutral","positive"]):
        i, _ = clean_tweet(tweet)
        i.append('\n')
        i = ["__label__"+tweet["class"]] + i
        inp.append(' '.join(i))

in_vocab = sorted(in_vocab)
out_vocab = sorted(out_vocab)
ttype_vocab = sorted(ttype_vocab)

In [33]:
inp[:10]

['__label__negative pakistan ghra tauq pakistan israel tasleem nahein kerta isko palestine kehta occupied palestine \n',
 '__label__negative madarchod mulle mathura nahi dikha tha jab mullo hindu iss liye mara lasse paise mag liye the \n',
 '__label__positive manya pradhan mantri mahoday shriman narendra modi pradhanmantri banne par hardik badhai tahe dil \n',
 '__label__positive jcb full trend chal rahi \n',
 '__label__positive ravishkumarblog loksabha janta sirf modi vote rahi thi kisi bjp without \n',
 '__label__negative bhosdike tum pechvade tatti rahoge \n',
 '__label__positive love bhaijan father son best pic from entire mashallah \n',
 '__label__negative tumhara pass abh deemagh hai nahi islea google apna deemagh banaya hua hai har koi tumhari tarh \n',
 '__label__positive weni ankere gae this weekend \n',
 '__label__negative lage raho mullo tumhre issi quran faad gyan kayal hain duniya allah bhi khus \n']

### Write to file in the fasttext format

In [34]:
with open("train_fast.txt","w",encoding='utf-8') as f:
    f.writelines(inp)